In [1]:
import pandas as pd
import io
from sklearn.model_selection import train_test_split
import sys

reload(sys) 
sys.setdefaultencoding('utf8')

NameError: name 'reload' is not defined

In [2]:
df = pd.read_csv(
    io.StringIO(
        open("urlset.csv", errors="ignore").read()), 
            error_bad_lines=False, 
#             escapechar='\\'
    )

disc = df.sample(frac=0.5)
gan =df[~df.index.isin(disc.index)]

gan_xtrain, gan_xtest, gan_ytrain, gan_ytest = train_test_split(
    gan.values[:, 1:12], gan.values[:, 13], test_size=.33)

disc_xtrain, disc_xtest, disc_ytrain, disc_ytest = train_test_split(
    disc.values[:, 1:12], disc.values[:, 13], )

disc = [disc_xtrain, disc_xtest, disc_ytrain, disc_ytest]
gan = [gan_xtrain, gan_xtest, gan_ytrain, gan_ytest]


# df=df.sample(df.shape[0])

b'Skipping line 18259: expected 14 fields, saw 15\nSkipping line 18273: expected 14 fields, saw 15\n'
C:\Users\codebrotherone\.conda\envs\cyber\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (1,2,3,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
import numpy as np

for col in df.columns.values[1:13]:

    df[col] = df[col].apply(lambda x: pd.to_numeric(x, errors='coerce'))
    df = df[~df[col].isnull()]

In [21]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split

# get data
data = df.values[:,1:]
x_train, x_test, y_train, y_test= train_test_split(data[:, :12], data[:, 12])

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_xtrain = scaler.fit_transform(x_train)


array([[1.00000000e+00, 1.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 8.57143000e-01, 6.45283000e-01],
       [1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 7.61397000e-01],
       [1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 6.00000000e-01],
       ...,
       [1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 8.30297000e-01],
       [3.83200000e-04, 1.00000000e+00, 1.42857143e-01, ...,
        0.00000000e+00, 5.16129000e-01, 0.00000000e+00],
       [9.35600000e-04, 1.00000000e+00, 1.42857143e-01, ...,
        6.81820000e-02, 6.47059000e-01, 7.77778000e-01]])

In [151]:
import io
import os
import numpy as np
import pandas as pd
from keras.layers import Input
from keras.models import Model, Sequential
from keras.layers.core import Dense, Dropout
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split


# You will use the Adam optimizer
def get_optimizer():
    return Adam(lr=0.0002, beta_1=0.5)


def create_gen_model():
    """This will create the generator model which will use tanh activation for output
    Notes:
        https://stackoverflow.com/questions/41489907/generative-adversarial-networks-tanh
    Args:
        input_dim (tuple): shape representing inputs to feed
        opt (keras.optimizers): optimizer to use (adam by default)
    Returns:
        generator (keras.SequentialModel): generator model
    """
    optimizer=get_optimizer()
    generator = Sequential()
    generator.add(Dense(128, input_dim=100))
    generator.add(LeakyReLU(0.2))

    generator.add(Dense(256))
    generator.add(LeakyReLU(0.2))

    generator.add(Dense(512))
    generator.add(LeakyReLU(0.2))

    generator.add(Dense(12, activation='tanh'))
    
    generator.compile(loss='binary_crossentropy', optimizer=opt)
    return generator


# discriminator
def create_disc_model():
    """This will create a discriminator model for our GAN network using Keras
    Args:
        Optimizer (keras.optimizers): Adam
    Returns:
        discriminator (keras.SequentialModel): discriminator model
    """
    optimizer=get_optimizer()
    discriminator = Sequential()
    # first layer
    discriminator.add(Dense(64, input_dim=12))
    discriminator.add(LeakyReLU(.2))
    discriminator.add(Dropout(.3))
    # second layer
    discriminator.add(Dense(128))
    discriminator.add(LeakyReLU(.2))
    discriminator.add(Dropout(.3))
    # third layer
    discriminator.add(Dense(64))
    discriminator.add(LeakyReLU(.2))
    discriminator.add(Dropout(.3))

    discriminator.add(Dense(1, activation='sigmoid'))
    discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return discriminator


def create_gan_network(discriminator, random_dim, generator, optimizer):
    # We initially set trainable to False since we only want to train either the
    # generator or discriminator at a time
    discriminator.trainable = False
    # gan input (noise) will be 100-dimensional vectors
    gan_input = Input(shape=(random_dim,))
    # the output of the generator (an image)
    x = generator(gan_input)
    # get the output of the discriminator (probability if the image is real or not)
    gan_output = discriminator(x)
    gan = Model(inputs=gan_input, outputs=gan_output)
    gan.compile(loss='binary_crossentropy', optimizer=optimizer)
    return gan


In [152]:

gan = create_gan_network(discriminator, random_dim, generator, adam)


In [156]:
from tqdm import tqdm 

# globals
epochs=1
batch_size=20
random_dim = 100

# Get the training and testing data
# for generator and discriminator
x_train = scaled_xtrain

# Split the training data into batches of size 128
batch_count = x_train.shape[0] / batch_size

# Build our GAN netowrk
adam = get_optimizer()
generator = create_gen_model()
discriminator = create_disc_model()


gan = create_gan_network(discriminator, random_dim, generator, adam)

for e in range(1, epochs+1):
    print('-'*15, 'Epoch %d' % e, '-'*15)
    for _ in tqdm(range(int(batch_count))):
        # Get a random set of input noise and images
        noise = np.random.normal(0, 1, size=[batch_size, random_dim])
        url_batch = x_train[np.random.randint(0, x_train.shape[0], size=batch_size)]

        # Generate fake url features
        gen_urls = generator.predict(noise)
        X = np.concatenate([url_batch, gen_urls])

        # Labels for generated and real data
        y_dis = np.zeros(2*batch_size)
        # One-sided label smoothing
        y_dis[:batch_size] = 1

        # Train discriminator or generator
        # based on below trainable attr()
        discriminator.trainable = True
        discriminator.train_on_batch(X, y_dis)

        # Train generator
        noise = np.random.normal(0, 1, size=[batch_size, random_dim])
        y_gen = np.ones(batch_size)
        gan.train_on_batch(noise, y_gen)

  0%|                                                                                         | 0/3596 [00:00<?, ?it/s]

--------------- Epoch 1 ---------------


C:\Users\codebrotherone\.conda\envs\cyber\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
100%|█████████████████████████████████████████████████████████████████████████████| 3596/3596 [00:25<00:00, 141.06it/s]


In [163]:
# Get a random set of input noise and images
noise = np.random.normal(0, 1, size=[batch_size, random_dim])
url_batch = x_train[np.random.randint(0, x_train.shape[0], size=batch_size)]
# Generate fake url features
gen_urls = generator.predict(noise)
X = np.concatenate([url_batch, gen_urls])

generator.fit(X, y_train)

ValueError: Error when checking input: expected dense_233_input to have shape (100,) but got array with shape (12,)

In [157]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split

# get data

data = df.values[:, 1:]

scaler = MinMaxScaler(feature_range=(0, 1))

x_train, x_test, y_train, y_test= train_test_split(data[:, :12], data[:, 12])


gan.evaluate(x_test, y_test, batch_size=20)

ValueError: Error when checking input: expected dense_233_input to have shape (100,) but got array with shape (12,)

In [155]:
discriminator.metrics_names

['loss', 'accuracy']

In [124]:
# Get a random set of input noise and images
noise = np.random.normal(0, 1, size=[batch_size, random_dim])
url_batch = x_train[np.random.randint(0, x_train.shape[0], size=batch_size)]

# Generate fake url features
gen_urls = generator.predict(noise)
X = np.concatenate([url_batch, gen_urls])

# Labels for generated and real data
y_dis = np.zeros(2*batch_size)
# One-sided label smoothing
y_dis[:batch_size] = 1

In [128]:
X.shape

(40,)

In [1]:
from sklearn.model_selection import train_test_split

def preprocess(df):
    """
    Takes dataframe and normalizes values between 0 and 1
    """
    from sklearn.preprocessing import normalize
    for col in df.columns.values[1:13]:
        df[col] = df[col].apply(lambda x: pd.to_numeric(x, errors='coerce'))
        df = df[~df[col].isnull()]
        df[col] = df[col].apply(lambda x: normalize(np.array([x])))
        
    
    normalized = normalize(np.array([df.iloc[:, 1:13].values]).reshape(df.shape, axis=1)
    
    return normalized
                           
def split_data(df):
    train_test_split(df)
                    

SyntaxError: invalid syntax (<ipython-input-1-6da2e4d573ea>, line 14)

In [33]:
df.values[:, 1:12]

array([[10000000.0, 1.0, 0.0, ..., 0.0, 0.0, 0.8],
       [10000000.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       [10000000.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       ...,
       [753.0, 1.0, 1.0, ..., 0.0, 0.0, 0.11111099999999999],
       [6.0, 1.0, 1.0, ..., 0.03025, 0.029144999999999997, 0.809735],
       [2547.0, 1.0, 1.0, ..., 0.017341, 0.020408000000000003, 0.636364]],
      dtype=object)

In [34]:
df.shape

(95914, 14)

In [ ]:
x, y = df.values()